In [38]:
import pandas as pd 
from scipy.stats import norm
import numpy as np 

In [39]:
time_period = '5_10'

In [40]:
data_FR = pd.read_csv('country-specific_results/results_pd_FR.csv',sep=';')
data_SW = pd.read_csv('country-specific_results/results_pd_SW.csv',sep=';')


In [41]:
data_SW = data_SW.rename(columns={'oddratio_ci':'final_odds_ratio_2_5_pd','oddratio_ci.1':'final_odds_ratio_5_10_pd'})

In [42]:
def prepare_df_AUS(df,country,time_period,nb_rows): 
    df['odds_ratio_'+time_period+'_'+country] = df['final_odds_ratio_'+time_period+'_pd']
    #df['odds_ratio_'+time_period+'_low'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[0])
    #df['odds_ratio_'+time_period+'_high'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[1][:-1])
    #df = df.replace('<0.001 ','0.0')
    #df = df.replace('<0.001','0.0')
    #df = df.astype(dtype={'odds_ratio_'+time_period+'_'+country:'float','odds_ratio_'+time_period+'_low'+'_'+country:'float','odds_ratio_'+time_period+'_high'+'_'+country:'float'})
    #counf_interval = norm.ppf(1 - 0.025/nb_rows)
    df['std_'+time_period+'_'+country] = df['std_'+time_period]
    counf_interval_95 = norm.ppf(1 - 0.025)

    df['odds_ratio_interval95_'+time_period+'_'+country] = df.apply(lambda x : str(x['odds_ratio_'+time_period+'_'+country])+'('+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])-counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])+counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+')',  axis=1)
    return df

def prepare_df_FR(df,country,time_period,nb_rows): 
    df['odds_ratio_'+time_period+'_'+country] = np.exp(df['coef_'+time_period])
    #df['odds_ratio_'+time_period+'_low'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[0])
    #df['odds_ratio_'+time_period+'_high'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[1][:-1])
    #df = df.replace('<0.001 ','0.0')
    #df = df.replace('<0.001','0.0')
    #df = df.astype(dtype={'odds_ratio_'+time_period+'_'+country:'float','odds_ratio_'+time_period+'_low'+'_'+country:'float','odds_ratio_'+time_period+'_high'+'_'+country:'float'})
    #counf_interval = norm.ppf(1 - 0.025/nb_rows)
    df['std_'+time_period+'_'+country] = df['std_'+time_period]
    counf_interval_95 = norm.ppf(1 - 0.025)

    df['odds_ratio_interval95_'+time_period+'_'+country] = df.apply(lambda x : str(x['odds_ratio_'+time_period+'_'+country])+'('+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])-counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])+counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+')',  axis=1)
    return df

def prepare_df_SW(df,country,time_period,nb_rows): 
    df = df.astype(dtype={'final_odds_ratio_'+time_period+'_pd':'str'})
    df['odds_ratio_'+time_period+'_'+country] = df['final_odds_ratio_'+time_period+'_pd'].apply(lambda x : x.split('(')[0])
    df['odds_ratio_'+time_period+'_low'+'_'+country] = df['final_odds_ratio_'+time_period+'_pd'].apply(lambda x : x.split('(')[1].split('-')[0])
    df['odds_ratio_'+time_period+'_high'+'_'+country] = df['final_odds_ratio_'+time_period+'_pd'].apply(lambda x : x.split('(')[1].split('-')[1][:-1])
    df = df.replace('<0.001 ','0.0')
    df = df.replace('<0.001','0.0')
    df = df.astype(dtype={'odds_ratio_'+time_period+'_'+country:'float','odds_ratio_'+time_period+'_low'+'_'+country:'float','odds_ratio_'+time_period+'_high'+'_'+country:'float'})
    counf_interval = norm.ppf(1 - 0.025/nb_rows)
    df['std_'+time_period+'_'+country] = (np.log(df['odds_ratio_'+time_period+'_high'+'_'+country])-np.log(df['odds_ratio_'+time_period+'_'+country]))/(counf_interval)
    counf_interval_95 = norm.ppf(1 - 0.025)

    df['odds_ratio_interval95_'+time_period+'_'+country] = df.apply(lambda x : str(x['odds_ratio_'+time_period+'_'+country])+' ('+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])-counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])+counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+')',  axis=1)
    return df
    

In [43]:
data_SW = prepare_df_SW(data_SW,'SW',time_period,54)
data_FR = prepare_df_FR(data_FR,'FR',time_period,1)

In [44]:
data_FR['n_'+time_period+'_FR2'] = data_FR['n_'+time_period+'_FR']

data_FR['n_'+time_period+'_FR'] = data_FR['n_'+time_period+'_FR']


df_final = pd.merge(data_FR,data_SW,on='code',how='outer')
#df_final = pd.merge(df_final,data_AUS,on='code',how='outer')
df_final = df_final.rename(columns={'medoc_y':'medoc'})

mapping = pd.read_csv('../helpers/mapping.csv',sep=';')
df_final = pd.merge(df_final,mapping,on='code',how='inner')
df_final['final_name'] = df_final['code']+': '+df_final['name_medoc']

In [45]:
def prepare_for_R(df,time_period): 
    for column in df.columns : 
        df = df.rename(columns={column: column.replace('_'+time_period,'')})
    #df['coeff_AUS'] = np.log(df['odds_ratio_AUS'])
    df['coeff_FR'] = np.log(df['odds_ratio_FR'])
    df['coeff_SW'] = np.log(df['odds_ratio_SW'])    
    return df
df_final = df_final[['code','final_name','odds_ratio_'+time_period+'_SW', 'std_'+time_period+'_SW',
            #'odds_ratio_'+time_period+'_AUS', 'std_'+time_period+'_AUS',
            'odds_ratio_'+time_period+'_FR', 'std_'+time_period+'_FR', 'n_'+time_period+'_FR','n_'+time_period+'_SW',
                     #'n_'+time_period+'_AUS',
                    ]]
for column in df_final.columns : 
        df_final = df_final.rename(columns={column: column.replace('_'+time_period,'')})
df_final_R = df_final.dropna().reset_index(drop=True)

df_final_R = prepare_for_R(df_final_R,time_period)
df_final_R.to_csv('temp/df_'+time_period+'_final.csv')
df_final_R

,code,final_name,odds_ratio_SW,std_SW,odds_ratio_FR,std_FR,n_FR,n_SW,coeff_FR,coeff_SW
0,A01,A01: STOMATOLOGICAL PREPARATIONS,1.40,0.026809,1.112456,0.007194,28.44,10.544236,0.106570,0.336472
1,A02,A02: DRUGS FOR ACID RELATED DISORDERS,1.18,0.019805,1.215756,0.006514,60.83,21.654127,0.195366,0.165514
2,A03,A03: DRUGS FOR FUNCTIONAL GASTROINTESTINAL DIS...,1.32,0.040602,1.174302,0.006703,37.44,4.162777,0.160674,0.277632
3,A04,A04: ANTIEMETICS AND ANTINAUSEANTS,1.23,0.123236,1.137234,0.012101,8.08,0.428360,0.128599,0.207014
4,A05,A05: BILE AND LIVER THERAPY,0.80,0.317530,1.341196,0.024418,2.05,0.060410,0.293561,-0.223144
5,A06,A06: DRUGS FOR CONSTIPATION,1.62,0.019835,1.548734,0.006751,40.48,20.423966,0.437438,0.482426
6,A07,A07: ANTIDIARRHEALS,1.07,0.042074,1.091798,0.007653,23.51,3.910154,0.087826,0.067659
7,A10,A10: DRUGS USED IN DIABETES,1.00,0.028776,1.079500,0.008936,15.68,9.121863,0.076498,0.000000
8,B01,B01: ANTITHROMBOTIC AGENTS,1.14,0.017992,1.124887,0.006548,41.40,35.059586,0.117683,0.131028
9,C01,C01: CARDIAC THERAPY,1.17,0.022375,1.121092,0.008789,16.61,14.289637,0.114303,0.157004


In [46]:
meta_analysis = pd.read_csv('temp/result_meta_analysis_R_'+time_period+'.csv')
meta_analysis = pd.merge(meta_analysis,df_final, on ='code',how='outer')


In [47]:
data_SW['n_'+time_period+'_SW'] = data_SW.apply(lambda x : str(x['n_brut_'+time_period+'_SW'])+' ('+str(np.round(x['n_'+time_period+'_SW'],2))+'%)',axis=1)
data_SW['n_'+time_period+'_SW_cont'] = data_SW.apply(lambda x : str(x['n_brut_'+time_period+'_SW_cont'])+' ('+str(np.round(x['n_'+time_period+'_SW_cont'],2))+'%)',axis=1)
data_SW_n = data_SW[['code','n_'+time_period+'_SW','n_'+time_period+'_SW_cont']]
data_FR['n_'+time_period+'_FR2'] = data_FR.apply(lambda x : str(x['n_brut_'+time_period+'_FR'])+' ('+str(np.round(x['n_'+time_period+'_FR2'],2))+'%)',axis=1)
data_FR['n_'+time_period+'_FR_cont'] = data_FR.apply(lambda x : str(x['n_brut_'+time_period+'_FR_cont'])+' ('+str(np.round(x['n_'+time_period+'_FR_cont'],2))+'%)',axis=1)
data_FR_n = data_FR[['code','n_'+time_period+'_FR2','n_'+time_period+'_FR_cont']]
data_n = pd.merge(data_SW_n,data_FR_n,on='code',how='outer')
data_n

,code,n_2_5_SW,n_2_5_SW_cont,n_2_5_FR2,n_2_5_FR_cont
0,A01,1920 (10.54%),13937 (7.65%),55613 (28.44%),50969 (26.07%)
1,A02,3943 (21.65%),34071 (18.71%),118942 (60.83%),108603 (55.54%)
2,A03,758 (4.16%),5737 (3.15%),73210 (37.44%),65368 (33.43%)
3,A04,78 (0.43%),626 (0.34%),15794 (8.08%),13888 (7.1%)
4,A05,11 (0.06%),133 (0.07%),4007 (2.05%),2974 (1.52%)
5,A06,3719 (20.42%),24918 (13.68%),79165 (40.48%),59094 (30.22%)
6,A07,712 (3.91%),6526 (3.58%),45970 (23.51%),42535 (21.75%)
7,A08,81 (0.44%),903 (0.5%),NaN,NaN
8,A09,77 (0.42%),591 (0.32%),NaN,NaN
9,A10,1661 (9.12%),16448 (9.03%),30670 (15.68%),28463 (14.56%)


In [48]:
meta_analysis['odd_ratio_meta'] = meta_analysis.apply(lambda x : str(np.round(x['odd_ratio'],2)) + ' ('+ str(np.round(x['low_odd_ratio'],2))+'-'+str(np.round(x['high_odd_ratio'],2))+')',axis=1)
for country in ['FR','SW']: 
    if country == 'AUS':
        nb_tests = data_AUS.shape[0]
    elif country =='FR':
        nb_tests = data_FR.shape[0]
    else:
        nb_tests = data_SW.shape[0]
    counf_interval_95 = norm.ppf(1 - 0.025/nb_tests)   
    meta_analysis['odds_ratio_'+country] = meta_analysis.apply(lambda x : str(np.round(x['odds_ratio_'+country],2))+' ('+str(np.exp(np.log(x['odds_ratio_'+country])-counf_interval_95*x['std_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+country])+counf_interval_95*x['std_'+country]).round(2))+')',  axis=1)

meta_analysis = meta_analysis[['code','odd_ratio_meta','p_value','odds_ratio_SW','odds_ratio_FR']]
meta_analysis

,code,odd_ratio_meta,p_value,odds_ratio_SW,odds_ratio_FR
0,A01,1.25 (0.86-1.81),5.554006e-02,1.4 (1.28-1.53),1.11 (1.09-1.14)
1,A02,1.2 (1.15-1.26),5.950699e-42,1.18 (1.11-1.26),1.22 (1.19-1.24)
2,A03,1.24 (1.02-1.49),2.566411e-04,1.32 (1.15-1.51),1.17 (1.15-1.2)
3,A04,1.14 (1.09-1.18),6.602098e-27,1.23 (0.82-1.85),1.14 (1.09-1.18)
4,A05,1.14 (0.52-2.49),5.808981e-01,0.8 (0.28-2.29),1.34 (1.24-1.46)
5,A06,1.58 (1.47-1.7),4.056154e-94,1.62 (1.52-1.73),1.55 (1.51-1.58)
6,A07,1.09 (1.06-1.12),5.289111e-31,1.07 (0.93-1.23),1.09 (1.06-1.12)
7,A08,nan (nan-nan),NaN,0.88 (0.6-1.3),nan (nan-nan)
8,A09,nan (nan-nan),NaN,1.29 (0.86-1.94),nan (nan-nan)
9,A10,1.04 (0.92-1.18),2.554586e-01,1.0 (0.91-1.1),1.08 (1.05-1.11)


In [49]:
pd.set_option('display.max_rows', 500)

mapping = pd.read_csv('../helpers/mapping.csv',sep=';')
meta_analysis = pd.merge(meta_analysis,data_n,on='code',how='outer')
meta_analysis = pd.merge(meta_analysis,mapping,on='code',how='inner')
meta_analysis['final_name'] = meta_analysis['code']+': '+ meta_analysis['name_medoc']

meta_analysis = meta_analysis[['final_name','code','odd_ratio_meta','p_value','odds_ratio_SW','odds_ratio_FR','n_'+time_period+'_SW','n_'+time_period+'_SW_cont','n_'+time_period+'_FR2','n_'+time_period+'_FR_cont']]
# Formater les p-values : arrondir à 3 décimales ou "< 0.001" si inférieur à 0.001
meta_analysis['p_value'] = meta_analysis['p_value'].apply(lambda x: '< 0.001' if x < 0.001 else round(x, 3))

meta_analysis = meta_analysis.replace('nan (nan-nan)','')
meta_analysis = meta_analysis.sort_values(by=['final_name'])
meta_analysis.to_csv('meta-analysis_result/meta_analysis_result_pd_final_'+time_period+'.csv',sep=';')